In [1]:
import os
import sys
sys.path.append(".")
sys.path.append("..")
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score, average_precision_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from feature_importance.scripts.simulations_util import *
from sklearn.linear_model import LinearRegression, ElasticNetCV
from scripts.competing_methods_local import *
from rbo_implementation import rbo_dict
import sys
sys.path.append('..')
sys.path.append('../..')
sys.path.append('.')
sys.path.append('./scripts')
from competing_methods_local import *
from simulations_util import *
from imodels.importance import RandomForestPlusRegressor as RandomForestPlusRegressorMDIPlus

/accounts/projects/binyu/zhongyuan_liang/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pd.read_csv("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_9978.csv")

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72
0,0.8,1.8,2.4,2.1,2.0,2.1,1.5,1.7,1.9,2.3,...,-15.50000,0.150000,10.670000,-1.560000,5795.000000,-12.100000,17.900000,10330.000000,-55.000000,0.00
1,2.8,3.2,3.3,2.7,3.3,3.2,2.9,2.8,3.1,3.4,...,-14.50000,0.480000,8.390000,3.840000,5805.000000,14.050000,29.000000,10275.000000,-55.000000,0.00
2,2.9,2.8,2.6,2.1,2.2,2.5,2.5,2.7,2.2,2.5,...,-15.90000,0.600000,6.940000,9.800000,5790.000000,17.900000,41.300000,10235.000000,-40.000000,0.00
3,4.7,3.8,3.7,3.8,2.9,3.1,2.8,2.5,2.4,3.1,...,-16.80000,0.490000,8.730000,10.540000,5775.000000,31.150000,51.700000,10195.000000,-40.000000,2.08
4,2.6,2.1,1.6,1.4,0.9,1.5,1.2,1.4,1.3,1.4,...,-10.51141,0.304716,9.872418,0.830116,5818.821222,10.511051,37.388335,10164.198442,-0.119949,0.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2529,0.3,0.4,0.5,0.5,0.2,0.3,0.4,0.4,1.3,2.2,...,-12.40000,0.070000,7.930000,-4.410000,5800.000000,-25.600000,21.800000,10295.000000,65.000000,0.00
2530,1.0,1.4,1.1,1.7,1.5,1.7,1.8,1.5,2.1,2.4,...,-12.00000,0.040000,5.950000,-1.140000,5845.000000,-19.400000,19.100000,10310.000000,15.000000,0.00
2531,0.8,0.8,1.2,0.9,0.4,0.6,0.8,1.1,1.5,1.5,...,-11.80000,0.060000,7.800000,-0.640000,5845.000000,-9.600000,35.200000,10275.000000,-35.000000,0.00
2532,1.3,0.9,1.5,1.2,1.6,1.8,1.1,1.0,1.9,2.0,...,-10.80000,0.250000,7.720000,-0.890000,5845.000000,-19.600000,34.200000,10245.000000,-30.000000,0.05


In [3]:
X_9978 = pd.read_csv("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361243.csv")
X_9978.shape

(1059, 72)

In [7]:
threshold = 0.95  # Adjust to 0.95 if needed

# Compute the correlation matrix
correlation_matrix = X_9978.corr()

# Identify features with high correlation
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
high_correlation_features = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

# Drop highly correlated features
X_9978_reduced = X_9978.drop(columns=high_correlation_features)
X_9978_reduced.to_csv("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_9978.csv", index=False)

print(f"Original number of features: {X_9978.shape[1]}")
print(f"Reduced number of features: {X_9978_reduced.shape[1]}")

Original number of features: 72
Reduced number of features: 47


In [ ]:

# Find duplicate columns by comparing values
duplicate_columns = []
columns_checked = set()

for col1 in df.columns:
    for col2 in df.columns:
        if col1 != col2 and col2 not in columns_checked:
            if df[col1].equals(df[col2]):
                duplicate_columns.append((col1, col2))
    columns_checked.add(col1)

# Print result
if duplicate_columns:
    print("Columns with identical values:")
    for col1, col2 in duplicate_columns:
        print(f"{col1} and {col2}")
else:
    print("No columns with identical values found.")


In [ ]:
df[]

In [ ]:
# check if there are identical columns in df
df = pd.read_csv("/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361243_original.csv")

# Find duplicate column names
duplicate_columns = df.columns[df.columns.duplicated()].tolist()

# Print result
if duplicate_columns:
    print("Duplicate columns found:")
    for col in duplicate_columns:
        print(col)
else:
    print("No duplicate columns found.")


In [ ]:
from sklearn import model_selection
def apply_splitting_strategy(X: np.ndarray,
                             y: np.ndarray,
                             splitting_strategy: str,
                             split_seed: str):
    if splitting_strategy in {'train-test-lowdata', 'train-tune-test-lowdata'}:
        test_size = 0.90  # X.shape[0] - X.shape[0] * 0.1
    elif splitting_strategy == "train-test":
        test_size = 0.33
    else:
        test_size = 0.2

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=split_seed)
    X_tune = None
    y_tune = None

    if splitting_strategy in {'train-tune-test', 'train-tune-test-lowdata'}:
        X_train, X_tune, y_train, y_tune = model_selection.train_test_split(
            X_train, y_train, test_size=0.2, random_state=split_seed)

    return X_train, X_tune, X_test, y_train, y_tune, y_test

In [ ]:
X = sample_real_data_X(source="openml",task_id=43)
y_1 = logistic_linear_model_random_feature(beta=1, s=5, frac_label_corruption=0.05, X=X, error_seed=42, feature_seed=1)
X_train, X_tune, X_test, y_train_1, y_tune, y_test_1 = apply_splitting_strategy(X, y_1, "train-test", 42)

In [ ]:
y_2 = logistic_linear_model_random_feature(beta=1, s=5, frac_label_corruption=0.25, X=X, error_seed=42, feature_seed=1)
X_train, X_tune, X_test, y_train_2, y_tune, y_test_2 = apply_splitting_strategy(X, y_2, "train-test", 42)

In [ ]:
np.all(y_train_1 == y_train_2)

In [ ]:
np.all(y_test_1 == y_test_2)

In [ ]:
print(y.sum())

In [ ]:
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 42)

In [ ]:
print(y_train.sum())

In [ ]:
print(y_test.sum())

In [ ]:
X, sample_indices = sample_real_data_X(source="openml", task_id=9978, sample_row_n=2000, seed = 0, return_sample_indices=True)
y,_,_ = sample_real_data_y(source="openml", task_id=9978, sample_row_n=2000, sample_indices=sample_indices, seed = 0)

In [ ]:
# from imodels.importance import RandomForestPlusRegressor

# rf_plus_model = RandomForestPlusRegressor()
# rf_plus_model.fit(X, y)
# mdi_plus_scores = rf_plus_model.get_mdi_plus_scores(X, y)

In [ ]:
from sklearn import model_selection
def apply_splitting_strategy(X: np.ndarray,
                             y: np.ndarray,
                             splitting_strategy: str,
                             split_seed: str):
    if splitting_strategy in {'train-test-lowdata', 'train-tune-test-lowdata'}:
        test_size = 0.90  # X.shape[0] - X.shape[0] * 0.1
    elif splitting_strategy == "train-test":
        test_size = 0.33
    else:
        test_size = 0.2

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=split_seed)
    X_tune = None
    y_tune = None

    if splitting_strategy in {'train-tune-test', 'train-tune-test-lowdata'}:
        X_train, X_tune, y_train, y_tune = model_selection.train_test_split(
            X_train, y_train, test_size=0.2, random_state=split_seed)

    return X_train, X_tune, X_test, y_train, y_tune, y_test

In [ ]:
def mask_unimportant_features(X, sorted_feature, mask, mask_values):
    array = copy.deepcopy(X)
    num_features = array.shape[1]
    num_masked = int(np.ceil(num_features * mask))
    
    # Select the least important features for each sample
    selected_indices = sorted_feature[:, num_masked:]

    for row_idx in range(array.shape[0]):
        for col_idx in selected_indices[row_idx]:
            if isinstance(mask_values[col_idx], (int, float, np.integer, np.floating)):
                array[row_idx, col_idx] = mask_values[col_idx]  # Replace with mean
            else:
                unique_vals = mask_values[col_idx]
                array[row_idx, col_idx] = unique_vals[1] if array[row_idx, col_idx] == unique_vals[0] else unique_vals[0]  # Replace with alternative value
    
    return num_masked, array

In [ ]:
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 0)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
mask_values = {}
for i in range(X_train.shape[1]):
    unique_values = np.unique(X_train[:, i])
    if len(unique_values) > 2:
        mask_values[i] = np.mean(X_train[:, i])
    else:
        mask_values[i] = list(unique_values)

In [ ]:
mask_values

In [ ]:
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_features="sqrt", random_state=0)
est.fit(X_train, y_train)

rf_plus_elastic = RandomForestPlusClassifier(rf_model=est, prediction_model=LogisticRegressionCV(penalty='elasticnet', l1_ratios=[0.1,0.5,0.99], solver = 'saga', cv=3, n_jobs=-1, tol=5e-4, max_iter=2000, random_state=0))
rf_plus_elastic.fit(X_train, y_train)


In [ ]:
# est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
# est.fit(X_train, y_train)

# rf_plus_elastic = RandomForestPlusRegressor(rf_model=est, prediction_model=ElasticNetCV(cv=3, l1_ratio=[0.1,0.5,0.99], max_iter=2000,random_state=0))
# rf_plus_elastic.fit(X_train, y_train)

# rf_plus_inbag = RandomForestPlusRegressor(rf_model=est, include_raw=False, fit_on="inbag", prediction_model=LinearRegression())
# rf_plus_inbag.fit(X_train, y_train, n_jobs=None)

# rf_plus_mdi_plus = RandomForestPlusRegressorMDIPlus(rf_model=est, prediction_model=ElasticNetCV(cv=3, l1_ratio=[0.1,0.5,0.99], max_iter=2000,random_state=0))
# rf_plus_mdi_plus.fit(X_train, y_train)

In [ ]:
local_fi_score_train_lmdi, _ = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_elastic, mode="absolute")
local_fi_score_train_shap, _ = tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=est, mode="absolute")
# local_fi_score_train_mdi_plus, _ = mdi_plus_evaluation_RF_retrain(X_train, y_train, X_test, fit=rf_plus_mdi_plus, mode="absolute")

In [ ]:
sorted_feature_train_lmdi = np.argsort(-local_fi_score_train_lmdi)
sorted_feature_train_shap = np.argsort(-local_fi_score_train_shap)
# sorted_feature_train_mdi_plus = np.argsort(-local_fi_score_train_mdi_plus)

# ablation_models = {"RF_Regressor": RandomForestRegressor(random_state=0)}
ablation_models = {"RF_Classifier": RandomForestClassifier(random_state=0)}

In [ ]:
mask_ratio = [0.1, 0.2, 0.3, 0.4]#, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for mask in mask_ratio:
    num_features_masked, X_train_masked = mask_unimportant_features(X_train, sorted_feature_train_lmdi, mask, mask_values)
    for a_model in ablation_models:
        ablation_models[a_model].fit(X_train_masked, y_train)
        y_pred = ablation_models[a_model].predict_proba(X_test)[:, 1]
        print(log_loss(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))

In [ ]:
mask_ratio = [0.1, 0.2, 0.3, 0.4]#, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for mask in mask_ratio:
    num_features_masked, X_train_masked = mask_unimportant_features(X_train, sorted_feature_train_shap, mask, mask_values)
    for a_model in ablation_models:
        ablation_models[a_model].fit(X_train_masked, y_train)
        y_pred = ablation_models[a_model].predict_proba(X_test)[:, 1]
        print(log_loss(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve

prob_true, prob_pred = calibration_curve(y_test, y_pred, n_bins=10)
plt.plot(prob_pred, prob_true, marker='o', label='Model')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray', label='Perfectly Calibrated')
plt.xlabel('Predicted Probability')
plt.ylabel('True Probability')
plt.title('Calibration Curve')
plt.legend()
plt.show()

In [ ]:
mask_ratio = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
for mask in mask_ratio:
    num_features_masked, X_train_masked = mask_unimportant_features(X_train, sorted_feature_train_mdi_plus, mask, mask_values)
    for a_model in ablation_models:
        ablation_models[a_model].fit(X_train_masked, y_train)
        y_pred = ablation_models[a_model].predict(X_test)
        print(log_loss(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))

In [ ]:
ablation_models[a_model].feature_importances_

In [ ]:
num_features_masked

In [ ]:
sorted_feature_train_mdi_plus[0]

In [ ]:
X_train_masked[:,4]

In [ ]:
assert False

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
mask_values = {}
for i in range(X_train.shape[1]):
    unique_values = np.unique(X_train[:, i])
    if len(unique_values) > 2:
        mask_values[i] = np.mean(X_train[:, i])
    else:
        mask_values[i] = list(unique_values)

In [ ]:
mask_values

In [ ]:
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
est.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import ElasticNetCV  
rf_plus_elastic = RandomForestPlusRegressor(rf_model=est, prediction_model=ElasticNetCV(cv=3, l1_ratio=[0.1,0.5,0.99], max_iter=2000,random_state=0))
rf_plus_elastic.fit(X_train, y_train)

In [ ]:
def LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    assert isinstance(fit, RandomForestPlusRegressor) or isinstance(fit, RandomForestPlusClassifier)
    rf_plus_mdi = RFPlusMDI(fit, mode = 'only_k', evaluate_on="all")
    local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
    local_fi_score_test = rf_plus_mdi.explain_linear_partial(X=X_test, y=None, ranking = True)
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

local_fi_score_train, _ = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_elastic, mode="absolute")

In [ ]:
local_fi_score_train[0]

In [ ]:
sorted_feature_train = np.argsort(-local_fi_score_train)

In [ ]:
sorted_feature_train[0]

In [ ]:
def mask_unimportant_features(X, sorted_feature, mask, mask_values):
    array = copy.deepcopy(X)
    num_features = array.shape[1]
    num_selected = int(np.ceil(num_features * mask))
    
    # Select the least important features for each sample
    selected_indices = sorted_feature[:, -1*num_selected:]

    for row_idx in range(array.shape[0]):
        for col_idx in selected_indices[row_idx]:
            if isinstance(mask_values[col_idx], (int, float)):
                array[row_idx, col_idx] = mask_values[col_idx]  # Replace with mean
            else:
                unique_vals = mask_values[col_idx]
                array[row_idx, col_idx] = unique_vals[1] if array[row_idx, col_idx] == unique_vals[0] else unique_vals[0]  # Replace with alternative value
    
    return num_selected, array

In [ ]:
mask_ratio = 0
num_features_selected, X_train_masked = mask_unimportant_features(X_train, sorted_feature_train, mask_ratio, mask_values)

In [ ]:
X_train_masked[0]

In [ ]:
X_train[0]

In [ ]:
X_train[0] == X_train_masked[0]

In [ ]:
# print the number of unique values in each column
X

In [ ]:
y.shape

##### Debug the differences yielded by AUROC and RBO

In [ ]:
X = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/X_361622.csv')
y = pd.read_csv('/accounts/projects/binyu/zhongyuan_liang/local_MDI+/imodels-experiments/feature_importance/data_openml/y_361622.csv')

In [ ]:
from sklearn import model_selection
def apply_splitting_strategy(X: np.ndarray,
                             y: np.ndarray,
                             splitting_strategy: str,
                             split_seed: str):
    if splitting_strategy in {'train-test-lowdata', 'train-tune-test-lowdata'}:
        test_size = 0.90  # X.shape[0] - X.shape[0] * 0.1
    elif splitting_strategy == "train-test":
        test_size = 0.33
    else:
        test_size = 0.2

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=split_seed)
    X_tune = None
    y_tune = None


    return X_train, X_tune, X_test, y_train, y_tune, y_test

In [ ]:
X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 2)

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=1)
est.fit(X_train, y_train)


In [ ]:
from sklearn.linear_model import ElasticNetCV
rf_plus_elastic = RandomForestPlusRegressor(rf_model=est, prediction_model=ElasticNetCV(cv=3, l1_ratio=[0.1,0.5,0.99], max_iter=2000,random_state=0))
rf_plus_elastic.fit(X_train, y_train)

rf_plus_elastic_no_parallel = RandomForestPlusRegressor(rf_model=est, prediction_model=ElasticNetCV(cv=3, l1_ratio=[0.1,0.5,0.99], max_iter=2000,random_state=0))
rf_plus_elastic_no_parallel.fit(X_train, y_train,n_jobs=None)

In [ ]:
def LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    assert isinstance(fit, RandomForestPlusRegressor) or isinstance(fit, RandomForestPlusClassifier)
    rf_plus_mdi = RFPlusMDI(fit, mode = 'only_k', evaluate_on="all")
    local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
    local_fi_score_test = rf_plus_mdi.explain_linear_partial(X=X_test, y=None, ranking = True)
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

lmdi_train, lmdi_test = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_elastic, mode="absolute")

In [ ]:
lmdi_train_no_parallel, lmdi_test_no_parallel = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_elastic_no_parallel, mode="absolute")

In [ ]:
np.allclose(lmdi_test, lmdi_test_no_parallel)

In [ ]:
lmdi_test_no_parallel

In [ ]:
from sklearn import model_selection
def apply_splitting_strategy(X: np.ndarray,
                             y: np.ndarray,
                             splitting_strategy: str,
                             split_seed: str):
    if splitting_strategy in {'train-test-lowdata', 'train-tune-test-lowdata'}:
        test_size = 0.90  # X.shape[0] - X.shape[0] * 0.1
    elif splitting_strategy == "train-test":
        test_size = 0.33
    else:
        test_size = 0.2

    X_train, X_test, y_train, y_test = model_selection.train_test_split(
        X, y, test_size=test_size, random_state=split_seed)
    X_tune = None
    y_tune = None


    return X_train, X_tune, X_test, y_train, y_tune, y_test

In [ ]:
X = sample_real_data_X(source="openml", task_id=146819)
y,_,_ = sample_real_data_y(source="openml", task_id=146819)
# y, support, beta = linear_model(X, sigma=None, s=5, beta=1, heritability=0.999999999999, return_support=True, seed=42)
# make y 0/1
# y = (y > 0).astype(int)
# y, support, beta = lss_model(X, m=3, r=2, beta=1, sigma=None, tau=0.5, heritability=0.99999999, return_support=True)
# y, support, beta = hierarchical_poly(X, m=3, r=2, beta=1, heritability=0.999999, return_support=True)
#y, support, beta = partial_linear_lss_model(X, s=1, m=3, r=2, beta=1, sigma=None, tau=0.5, heritability=0.99999999, return_support=True)

In [ ]:
X = sample_real_data_X(source="openml", task_id=9978, normalize=True)
y, support,_ = logistic_poly_random_feature(X, m=3,r=2, beta=1,frac_label_corruption=0,error_seed=3, feature_seed=1, return_support=True)

X_train, X_tune, X_test, y_train, y_tune, y_test = apply_splitting_strategy(X, y, "train-test", 1)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Fitting Models")
# fit RF model
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=0)
est.fit(X_train, y_train)

rf_plus_baseline = RandomForestPlusRegressor(rf_model=est,
                                             prediction_model=LinearRegression(),
                                             fit_on="inbag",
                                             include_raw=False)
# rf_plus_baseline.fit(X_train, y_train)
# assert np.allclose(rf_plus_baseline.predict(X_train),est.predict(X_train))

In [ ]:
from sklearn.linear_model import ElasticNetCV
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)

rf_plus_elastic = RandomForestPlusClassifier(rf_model=est, prediction_model=LogisticRegressionCV(penalty='elasticnet', l1_ratios=[0.1,0.5,0.9,0.99], solver = 'saga', cv=3, n_jobs=-1, tol=5e-4, max_iter=2000, random_state=0))
rf_plus_elastic.fit(X_train, y_train)

In [ ]:
def LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    assert isinstance(fit, RandomForestPlusRegressor) or isinstance(fit, RandomForestPlusClassifier)
    rf_plus_mdi = RFPlusMDI(fit, mode = 'only_k', evaluate_on="all")
    local_fi_score_train = rf_plus_mdi.explain_linear_partial(X=X_train, y=y_train, ranking = True)
    local_fi_score_test = rf_plus_mdi.explain_linear_partial(X=X_test, y=None, ranking = True)
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

lmdi_train, lmdi_test = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_elastic, mode="absolute")

In [ ]:
def tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=None, mode="absolute"):
    """
    Compute average treeshap value across observations.
    Larger absolute values indicate more important features.
    :param X: design matrix
    :param y: response
    :param fit: fitted model of interest (tree-based)
    :return: dataframe of shape: (n_samples, n_features)
    """
    explainer = shap.TreeExplainer(fit)
    local_fi_score_train = explainer.shap_values(X_train, check_additivity=False)
    local_fi_score_test = explainer.shap_values(X_test, check_additivity=False)
    if sklearn.base.is_classifier(fit):
        if mode == "absolute":
            return np.abs(local_fi_score_train[:,:,1]), np.abs(local_fi_score_test[:,:,1])
        else:
            return local_fi_score_train[:,:,1], local_fi_score_test[:,:,1]
    if mode == "absolute":
        return np.abs(local_fi_score_train), np.abs(local_fi_score_test)
    else:
        return local_fi_score_train, local_fi_score_test

In [ ]:
lmdi_train, lmdi_test = LFI_evaluation_MDIRFPlus_all_ranking_retrain(X_train, y_train, X_test, fit=rf_plus_elastic, mode="absolute")

In [ ]:
shap_train, shap_test = tree_shap_evaluation_RF_retrain(X_train, y_train, X_test, fit=est, mode="absolute")

In [ ]:
temp = []
for i in range(lmdi_test.shape[0]):
    temp.append(roc_auc_score(support, lmdi_test[i, :]))
print(np.mean(temp))

In [ ]:
temp = []
for i in range(shap_test.shape[0]):
    temp.append(roc_auc_score(support, shap_test[i, :]))
print(np.mean(temp))

In [ ]:
lasso_train, lasso_test =

In [ ]:
y

In [ ]:
X.shape

In [ ]:
y

In [ ]:
np.shape(y)

In [ ]:
math.ceil(0.05*len(y))

In [ ]:
support

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, random_state=0)
est = RandomForestClassifier(n_estimators=100, min_samples_leaf=3, max_features='sqrt', random_state=42)
est.fit(X_train, y_train)
rf_plus_base = RandomForestPlusClassifier(rf_model=est)
rf_plus_base.fit(X_train, y_train)
# rf_plus_base_oob = RandomForestPlusRegressor(rf_model=est, fit_on="oob")
# rf_plus_base_oob.fit(X_train, y_train)

In [ ]:
explainer = RFPlusLime(rf_plus_base)
local_fi_score_train_subset = explainer.explain(X_train, X_train[:20])

In [ ]:
def ground_truth_fi_derivation(X, support, dgp):
    fi = np.zeros_like(X)
    assert dgp == "linear"
    fi = np.abs(X) 
    fi[:, support == 0] = 0
    return fi

In [ ]:
X_train[0]

In [ ]:
temp = ground_truth_fi_derivation(X_train, support, "linear")[0]
temp

In [ ]:
def encode_largest_k(arr, k):
    indices = np.argpartition(arr, -k)[-k:]
    encoded_array = np.zeros_like(arr)
    encoded_array[indices] = 1
    return encoded_array

In [ ]:
encode_largest_k(temp, 1)

In [ ]:
local_fi_score_train_subset

In [ ]:
local_fi_score_train_subset[:,:,1].shape

In [ ]:
local_fi_score_train_subset[:,:,0][0]

In [ ]:
local_fi_score_train_subset.shape

In [ ]:
local_fi_score_train_subset[:,:,1][1]*2

In [ ]:
np.sum(np.abs(local_fi_score_train_subset[1]),axis=-1)

In [ ]:
rf_plus_kernel_shap = RFPlusKernelSHAP(rf_plus_base)
local_fi_score_train = None
local_fi_score_train_subset = rf_plus_kernel_shap.explain(X_train=X_train, X_test=X_train)

In [ ]:
local_fi_score_train_subset.shape

In [ ]:
np.sum(np.abs(local_fi_score_train_subset),axis=-1).shape

In [ ]:
local_fi_score_train_subset[:,:,0][0]

In [ ]:
local_fi_score_train_subset[:,:,1][0]

In [ ]:
alo_mdi = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")
rf_plus_lime = RFPlusLime(rf_plus_base)

In [ ]:
local_fi_score_train_l2_norm_sign = np.abs(alo_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True, sign=True))
local_fi_score_train_l2_norm = np.abs(alo_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=True))
local_fi_score_train = np.abs(alo_mdi.explain_linear_partial(X=X_train, y=y_train, l2norm=False))
lime_train = np.abs(rf_plus_lime.explain(X_train=X_train, X_test=X_train).values)

In [ ]:
auroc_lmdi_norm_sign= []
for i in range(local_fi_score_train.shape[0]):       
    auroc_lmdi_norm_sign.append(roc_auc_score([1]*5+[0]*5, local_fi_score_train_l2_norm_sign[i]))
print(np.mean(auroc_lmdi_norm_sign))

In [ ]:
auroc_lmdi_norm= []
for i in range(local_fi_score_train.shape[0]):       
    auroc_lmdi_norm.append(roc_auc_score([1]*5+[0]*5, local_fi_score_train_l2_norm[i]))
print(np.mean(auroc_lmdi_norm))

In [ ]:
auroc_lmdi= []
for i in range(local_fi_score_train.shape[0]):       
    auroc_lmdi.append(roc_auc_score([1]*5+[0]*5, local_fi_score_train[i]))
print(np.mean(auroc_lmdi))

In [ ]:
auroc_lime= []
for i in range(local_fi_score_train.shape[0]):       
    auroc_lime.append(roc_auc_score([1]*5+[0]*5, lime_train[i]))
print(np.mean(auroc_lime))

In [ ]:
### LIME assessment
temp_lime = []
for i in range(5):
    indices_correct = np.argwhere(np.array(auroc_lime) == 1.0).flatten()
    indices = np.argwhere((-1 * lime_train).argsort() == i)[:,1][indices_correct]
    values = X_train[indices_correct][np.arange(indices_correct.shape[0]), indices]
    mean_abs_values = np.mean(np.abs(values))
    temp_lime.append(mean_abs_values)
print(temp_lime)

In [ ]:
temp_lmdi = []
for i in range(5):
    indices_correct = np.argwhere(np.array(auroc_lmdi_norm) == 1.0).flatten()
    indices = np.argwhere((-1 * local_fi_score_train_l2_norm).argsort() == i)[:,1][indices_correct]
    values = X_train[indices_correct][np.arange(indices_correct.shape[0]), indices]
    mean_abs_values = np.mean(np.abs(values))
    temp_lmdi.append(mean_abs_values)
print(temp_lmdi)

In [ ]:
# plot temp_lmdi and temp_lime
import matplotlib.pyplot as plt
plt.plot(temp_lmdi, label="lmdi")
plt.plot(temp_lime, label="lime")
plt.legend()
plt.show()

In [ ]:
np.argwhere((-1 * local_fi_score_train).argsort() == 0)[:,1]

In [ ]:
indices_correct

In [ ]:
indices_correct = np.argwhere(np.array(auroc_lmdi) == 1.0)
indices = np.argwhere((-1*local_fi_score_train).argsort() == 0)
values = [X_train[tuple(indices[index])] for index in indices_correct]
np.mean(np.abs(values))

In [ ]:
indices_correct = np.argwhere(np.array(auroc_lmdi) == 1.0)
indices = np.argwhere((-1*local_fi_score_train).argsort() == 1)
values = [X_train[tuple(indices[index])] for index in indices_correct]
np.mean(np.abs(values))

In [ ]:
indices_correct = np.argwhere(np.array(auroc_lmdi) == 1.0)
indices = np.argwhere((-1*local_fi_score_train).argsort() == 2)
values = [X_train[tuple(indices[index])] for index in indices_correct]
np.mean(np.abs(values))

In [ ]:
indices_correct = np.argwhere(np.array(auroc_lmdi) == 1.0)
indices = np.argwhere((-1*local_fi_score_train).argsort() == 3)
values = [X_train[tuple(indices[index])] for index in indices_correct]
np.mean(np.abs(values))

In [ ]:
indices_correct = np.argwhere(np.array(auroc_lmdi) == 1.0)
indices = np.argwhere((-1*local_fi_score_train).argsort() == 4)
values = [X_train[tuple(indices[index])] for index in indices_correct]
np.mean(np.abs(values))

In [ ]:
for index in indices_correct:
    print(tuple(indices[index]))

In [ ]:
indices = np.argwhere(np.array(auroc_lmdi) == 1.0)

In [ ]:
indices.shape

In [ ]:
indices = np.argwhere((-1*local_fi_score_train_l2_norm).argsort() == 1)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*local_fi_score_train_l2_norm).argsort() == 2)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*local_fi_score_train_l2_norm).argsort() == 3)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*local_fi_score_train_l2_norm).argsort() == 4)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*lime_train).argsort() == 0)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*lime_train).argsort() == 1)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*lime_train).argsort() == 2)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*lime_train).argsort() == 3)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
indices = np.argwhere((-1*lime_train).argsort() == 4)
values = [X_train[tuple(index)] for index in indices]
np.mean(np.abs(values))

In [ ]:
# Find mean of X_train of all index with 0 in lime_train


In [ ]:
local_fi_score_train

In [ ]:
rf_plus_mdi_train = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="all")
rf_plus_mdi_test = RFPlusMDI(rf_plus_base_oob, evaluate_on="all")
local_fi_score_train_lmdi_plus_method2 = np.abs(rf_plus_mdi_train.explain_linear_partial(X=X_train, y=y_train, leaf_average=False))
local_fi_score_test_lmdi_plus_method2 = np.abs(rf_plus_mdi_test.explain_linear_partial(X=X_test, y=None))
local_fi_score_train_lmdi_plus_method2_l2_norm = np.abs(rf_plus_mdi_train.explain_linear_partial(X=X_train, y=y_train, l2norm=True, leaf_average=False))
local_fi_score_test_lmdi_plus_method2_l2_norm = np.abs(rf_plus_mdi_test.explain_linear_partial(X=X_test, y=None, l2norm=True))

In [ ]:
explainer = shap.TreeExplainer(est)
local_fi_score_train_shap = np.abs(explainer.shap_values(X_train, check_additivity=False))
local_fi_score_test_shap = np.abs(explainer.shap_values(X_test, check_additivity=False))

In [ ]:
local_fi_score_train_shap

In [ ]:
auroc_shap = []
rbo_lst_09_shap = []
num_captured_shap = []
for i in range(local_fi_score_train_shap.shape[0]):
    fi_data_i = local_fi_score_train_shap[i]
    ground_truth_fi_i = np.abs(X_train)[i]
    ground_truth_fi_i[support == 0] = 0
    dict_predictions = dict(enumerate(fi_data_i))
    dict_ground_truth = dict(enumerate(ground_truth_fi_i))      
    num_signal_features = int(np.sum(support))            
    auroc_shap.append(roc_auc_score(support, fi_data_i))
    rbo_lst_09_shap.append(rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=0.9)[2])
    sorted_indices = np.argsort(-fi_data_i)
    top_indices = sorted_indices[:num_signal_features]
    actual_signal_features = np.sum(support[top_indices])
    num_captured_shap.append(actual_signal_features)

In [ ]:
auroc_lmdi_plus= []
rbo_lst_09_lmdi_plus = []
num_captured_lmdi_plus = []
for i in range(local_fi_score_train_lmdi_plus_method2.shape[0]):
    fi_data_i = local_fi_score_train_lmdi_plus_method2[i]
    ground_truth_fi_i = np.abs(X_train)[i]
    ground_truth_fi_i[support == 0] = 0
    dict_predictions = dict(enumerate(fi_data_i))
    dict_ground_truth = dict(enumerate(ground_truth_fi_i))      
    num_signal_features = int(np.sum(support))            
    auroc_lmdi_plus.append(roc_auc_score(support, fi_data_i))
    rbo_lst_09_lmdi_plus.append(rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=0.9)[2])
    sorted_indices = np.argsort(-fi_data_i)
    top_indices = sorted_indices[:num_signal_features]
    actual_signal_features = np.sum(support[top_indices])
    num_captured_lmdi_plus.append(actual_signal_features)

In [ ]:
auroc_lmdi_plus_avg= []
rbo_lst_09_lmdi_plus_avg = []
num_captured_lmdi_plus_avg = []
for i in range(local_fi_score_train_lmdi_plus_method2_l2_norm.shape[0]):
    fi_data_i = local_fi_score_train_lmdi_plus_method2_l2_norm[i]
    ground_truth_fi_i = np.abs(X_train)[i]
    ground_truth_fi_i[support == 0] = 0
    dict_predictions = dict(enumerate(fi_data_i))
    dict_ground_truth = dict(enumerate(ground_truth_fi_i))      
    num_signal_features = int(np.sum(support))            
    auroc_lmdi_plus_avg.append(roc_auc_score(support, fi_data_i))
    rbo_lst_09_lmdi_plus_avg.append(rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=0.9)[2])
    sorted_indices = np.argsort(-fi_data_i)
    top_indices = sorted_indices[:num_signal_features]
    actual_signal_features = np.sum(support[top_indices])
    num_captured_lmdi_plus_avg.append(actual_signal_features)

In [ ]:
print(np.array(auroc_shap).mean(), np.array(rbo_lst_09_shap).mean(), np.array(num_captured_shap).mean())
print(np.array(auroc_lmdi_plus).mean(), np.array(rbo_lst_09_lmdi_plus).mean(), np.array(num_captured_lmdi_plus).mean())
print(np.array(auroc_lmdi_plus_avg).mean(), np.array(rbo_lst_09_lmdi_plus_avg).mean(), np.array(num_captured_lmdi_plus_avg).mean())

In [ ]:
auroc_shap[5], rbo_lst_09_shap[5]

In [ ]:
auroc_lmdi_plus[5], rbo_lst_09_lmdi_plus[5]

In [ ]:
ground_truth_fi_i = np.abs(X_test)[5]
ground_truth_fi_i[support == 0] = 0
ground_truth_fi_i

In [ ]:
p = 0.9

In [ ]:
dict_predictions = dict(enumerate(local_fi_score_test_shap[5]))
dict_ground_truth = dict(enumerate(ground_truth_fi_i))
rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=p, verbose=True)[2]

In [ ]:
local_fi_score_test_lmdi_plus[5]

In [ ]:
dict_predictions = dict(enumerate(local_fi_score_test_lmdi_plus[5]))
dict_ground_truth = dict(enumerate(ground_truth_fi_i))
rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=p, verbose=True)[2]

In [ ]:
temp = np.array([0.30647695  , 0.17410994, 0.816055, 0.17842848, 0.10012125,
       0.26276102, 0.26671546, 0.28039733, 0.23719995, 0.25739759])

In [ ]:
dict_predictions = dict(enumerate(temp))#local_fi_score_test_lmdi_plus[5]))
dict_ground_truth = dict(enumerate(ground_truth_fi_i))
rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=p, verbose=True)[2]

##### Debug two group setting with intercept

In [ ]:
X = sample_normal_X_subgroups(n = 500, d=10, mean= [[0]*10,[0]*5+[0]*5], scale =[[1]*10,[1]*10])
temp = linear_model(X, beta=1, sigma=None, heritability=0.6, s=5, return_support=True)
y = temp[0]
support = temp[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
linear_model(X, beta=1, sigma=None, heritability=0.2, s=5, return_support=True)[1]

In [ ]:
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)

# rf_plus_base_oob = RandomForestPlusRegressor(rf_model=rf, fit_on="oob")
# rf_plus_base_oob.fit(X_train, y_train)

# rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=rf, include_raw=False, fit_on="inbag", prediction_model=Ridge(alpha=1e-6))
# rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
np.random.seed(42)
if X_train.shape[0] > 100:
    indices_train = np.random.choice(X_train.shape[0], 100, replace=False)
    X_train_subset = X_train[indices_train]
    y_train_subset = y_train[indices_train]
else:
    indices_train = np.arange(X_train.shape[0])
    X_train_subset = X_train
    y_train_subset = y_train

if X_test.shape[0] > 100:
    indices_test = np.random.choice(X_test.shape[0], 100, replace=False)
    X_test_subset = X_test[indices_test]
    y_test_subset = y_test[indices_test]
else:
    indices_test = np.arange(X_test.shape[0])
    X_test_subset = X_test
    y_test_subset = y_test

In [ ]:
indices_train_0 = np.where(X_train_subset[:, -1] == 0)[0]
indices_test_0 = np.where(X_test_subset[:, -1] == 0)[0]

In [ ]:
indices_train_1 = np.where(X_train_subset[:, -1] == 1)[0]
indices_test_1 = np.where(X_test_subset[:, -1] == 1)[0]

In [ ]:
local_fi_score_train, local_fi_score_train_subset, local_fi_score_test, local_fi_score_test_subset = tree_shap_evaluation_RF(X_train=X_train, y_train=y_train, X_train_subset = X_train_subset, y_train_subset=y_train_subset,X_test=X_test, y_test=y_test, X_test_subset=X_test_subset, y_test_subset=y_test_subset,fit=rf, mode="absolute")

In [ ]:
new_support_train = np.abs(X_train_subset)
new_support_test = np.abs(X_test)
new_support_train[:, -5:] = 0
new_support_test[:, -5:] = 0

In [ ]:
data = local_fi_score_train_subset
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print("Treeshap Trainsubset")
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
data = local_fi_score_test 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print("Treeshap Test")
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
def rank_biased_overlap(list1, list2, p=0.9):
    """
    Compute the Rank-Biased Overlap (RBO) between two ranked lists.

    Parameters:
    - list1: numpy array or list of the first ranked list
    - list2: numpy array or list of the second ranked list
    - p: the discount factor (default is 0.9, which is commonly used)

    Returns:
    - rbo: the Rank-Biased Overlap score
    """

    # Convert lists to numpy arrays if they're not already
    list1 = np.asarray(list1)
    list2 = np.asarray(list2)

    # Get the indices that would sort the arrays in descending order
    sorted_indices1 = np.argsort(-list1)
    sorted_indices2 = np.argsort(-list2)

    # Rank lists based on sorted indices
    ranked_list1 = sorted_indices1
    ranked_list2 = sorted_indices2

    # Initialize the overlap
    overlap = 0.0
    min_len = min(len(ranked_list1), len(ranked_list2))
    
    # Compute the RBO
    for i in range(min_len):
        # Calculate the overlap at rank i
        rank_i_overlap = len(set(ranked_list1[:i+1]) & set(ranked_list2[:i+1]))
        
        # Add the discounted overlap to the total
        overlap += (rank_i_overlap / (i + 1)) * (p ** (i + 1))
    
    # Normalize the score
    normalization = (1 - p) / (1 - p ** (min_len + 1))
    rbo = overlap * normalization
    
    return rbo

In [ ]:
data = local_fi_score_train_subset
rbo = []
for i in range(data.shape[0]):
        rbo.append(rank_biased_overlap(new_support_train[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
data = local_fi_score_test
rbo = []
for i in range(data.shape[0]):
        rbo.append(rank_biased_overlap(new_support_test[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
rf_plus_mdi_train = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")
rf_plus_mdi_test = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
local_fi_score_train = np.abs(rf_plus_mdi_train.explain_subtract_intercept(X=X_train, y=y_train))
local_fi_score_test = np.abs(rf_plus_mdi_test.explain_subtract_intercept(X=X_test, y=None))
local_fi_score_test_subset = np.abs(rf_plus_mdi_test.explain_subtract_intercept(X=X_test_subset, y=None))
local_fi_score_train_subset = local_fi_score_train[indices_train]

In [ ]:
data = local_fi_score_train_subset 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
data = local_fi_score_test 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
new_support_train = np.abs(X_train_subset)
new_support_test = np.abs(X_test)
new_support_train[:, -5:] = 0
new_support_test[:, -5:] = 0

In [ ]:
data = local_fi_score_train_subset
rbo = []
for i in range(data.shape[0]):
        rbo.append(rank_biased_overlap(new_support_train[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
data = local_fi_score_test
rbo_lst = []
for i in range(data.shape[0]):
        rbo_lst.append(rbo.RankingSimilarity(new_support_test[i], data[i]).rbo())#rbo.append(rank_biased_overlap(new_support_test[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
new_support_test[0]

In [ ]:
data[0]

In [ ]:
rbo.RankingSimilarity(S, T).rbo()

In [ ]:
rf_plus_mdi_train = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")
rf_plus_mdi_test = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
local_fi_score_train = np.abs(rf_plus_mdi_train.explain(X=X_train, y=y_train)[1])
local_fi_score_test = np.abs(rf_plus_mdi_test.explain(X=X_test, y=None)[1])
local_fi_score_test_subset = np.abs(rf_plus_mdi_test.explain(X=X_test_subset, y=None)[1])
local_fi_score_train_subset = local_fi_score_train[indices_train]

In [ ]:
data = local_fi_score_train_subset 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
data = local_fi_score_test 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())